#### Imports

In [1]:
import pandas as pd 
import re
from pathlib import Path


In [2]:
data_tim = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_tim_copy.csv')
data_sien = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_sien_copy.csv')

data = pd.concat([data_sien, data_tim])


In [3]:
data

,entry_num,Time (s),X (m/s^2),Y (m/s^2),Z (m/s^2),Time (s).1,X (rad/s),Y (rad/s),Z (rad/s),name_climber,grading,num_attempt,fall_top,climbers_rating
0,15,0.202257,0.616807,0.545096,-0.119170,0.202257,-0.738735,-0.693665,-0.053444,sien,4c,1,0,easy
1,15,0.403552,0.427085,-0.449999,0.075464,0.403552,-0.314339,-0.378076,0.015612,sien,4c,1,0,easy
2,15,0.604846,0.570240,0.152899,-0.213106,0.604846,-0.273812,-0.442562,-0.130227,sien,4c,1,0,easy
3,15,0.806141,0.295633,-0.453686,0.244799,0.806141,-0.260747,-0.114135,-0.019194,sien,4c,1,0,easy
4,15,1.007435,0.206272,-0.927811,-0.512616,1.007435,-0.160211,-0.226173,-0.211577,sien,4c,1,0,easy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,23,93.603675,0.198353,-1.241287,-1.830433,93.603675,0.068989,-0.352130,0.652643,tim,5c,1,1,hard
7664,23,93.804975,1.519217,1.315391,-0.367484,93.804975,-0.284642,-1.725383,-0.025087,tim,5c,1,1,hard
7665,23,94.006275,0.730766,0.629014,-1.913436,94.006275,-1.022073,-1.979728,0.774434,tim,5c,1,1,hard
7666,23,94.207576,0.974350,-0.727642,-1.214439,94.207576,-0.439935,-1.797494,0.093850,tim,5c,1,1,hard


#### Add datetime to df

Get datetime from phyphox metadata

In [4]:
raw_directory = Path('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/climb_data_phyphox')

# subdirectories 
subdir_sien = raw_directory.joinpath('sien')
subdir_tim = raw_directory.joinpath('tim')

# datetime 
datetime_dict = {}

# get start & end datetimes for each climb entry 
for climber in (subdir_sien, subdir_tim):
    for file_path in climber.iterdir():

        if file_path.name == '.DS_Store': continue 
        # get climb entry number
        entry_num = int(re.findall(r'\d+', file_path.name)[0])
        # dirs for each climb
        climbdir = raw_directory.joinpath(file_path)
        # metadata dir 
        metadir = climbdir.joinpath('meta')
        # open time file 
        with open(metadir.joinpath('time.csv'), 'r') as datetime_file:
            # put start & end time into dict 
            datetime_df = pd.read_csv(datetime_file)
            start_time, end_time = datetime_df['system time text'][0], datetime_df['system time text'][1]
            datetime_dict[entry_num] = (start_time, end_time)

print(datetime_dict)

{15: ('2023-06-07 16:36:31.299 UTC+02:00', '2023-06-07 16:38:32.531 UTC+02:00'), 4: ('2023-06-07 13:45:35.533 UTC+02:00', '2023-06-07 13:48:47.258 UTC+02:00'), 5: ('2023-06-07 13:53:48.058 UTC+02:00', '2023-06-07 13:56:49.460 UTC+02:00'), 14: ('2023-06-07 16:31:16.179 UTC+02:00', '2023-06-07 16:32:59.567 UTC+02:00'), 25: ('2023-06-07 18:05:25.347 UTC+02:00', '2023-06-07 18:08:21.544 UTC+02:00'), 2: ('2023-06-07 13:28:35.606 UTC+02:00', '2023-06-07 13:30:21.669 UTC+02:00'), 13: ('2023-06-07 16:28:15.035 UTC+02:00', '2023-06-07 16:29:26.392 UTC+02:00'), 12: ('2023-06-07 16:18:55.545 UTC+02:00', '2023-06-07 16:22:44.271 UTC+02:00'), 3: ('2023-06-07 13:37:23.658 UTC+02:00', '2023-06-07 13:39:58.248 UTC+02:00'), 24: ('2023-06-07 17:58:33.461 UTC+02:00', '2023-06-07 18:03:18.690 UTC+02:00'), 1: ('2023-06-07 13:12:51.871 UTC+02:00', '2023-06-07 13:15:00.053 UTC+02:00'), 17: ('2023-06-07 16:57:12.188 UTC+02:00', '2023-06-07 17:01:05.538 UTC+02:00'), 16: ('2023-06-07 16:43:36.219 UTC+02:00', '2

In [5]:

# creates & maps datetime sequences for each climb entry 
datetime_sequence_dict = {}
for group_name, group_data in data.groupby('entry_num'):
    # create & save datetime sequence for each climb entry 
    start_datetime, end_datetime = pd.to_datetime(datetime_dict[group_name][0]), pd.to_datetime(datetime_dict[group_name][1])
    datetime_range = pd.date_range(start=start_datetime, end=end_datetime, periods=len(group_data))
    datetime_sequence_dict[group_name] = datetime_range

# inits null datetime var
if not 'datetime' in data:
    data.insert(1, 'datetime', float('nan'))

# fills in datetime sequences 
for entry_id in data['entry_num'].unique():
    mask = data['entry_num'] == entry_id
    data.loc[mask, 'datetime'] = datetime_sequence_dict[entry_id]

# check 
data['datetime'].isna().sum()


0

#### cutting falls from data

#### EDA - table

In [10]:
# code from DMT
# datatypes - NOTE: subject to change (only accelerometer & gyrometer rn)
types = ['id','datetime', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric'] 

# nb of values 
n_values = [data[var].count() for var in data]

# nb of unique values
n_unique = [data[var].nunique() for var in data]


# TODO: do once we have the final variables (eg. for grade)
## mean + std 
# mean_std = []

# for var in data: 
#     if var not in ['datetime', 'entry_num']: # TODO 
#         mean = round(data[var].mean(), 2)
#         std = round(data[var].std(), 2) 
#         res = f'{mean} ({std})'
#         mean_std.append(res)
# mean_std.insert(0,'Na')
# mean_std.insert(0,'Na')

# # range of values
# val_range = []
# for var in data:
#     if var != 'id':
#         r = (data[var].min(), data[var].max())
#         if var != 'time':
#             r = (round(r[0]), round(r[1]))
#         else:
#             r = (data['time'].dt.date.min(), data['time'].dt.date.max())
#         val_range.append(r)
#         r = data['time'].dt.date
# val_range.insert(0, 'Na')

# # date range 
# time_range = []
# for var in data: 
#     # if var != 'id':
#     grouped = data.groupby(var)['time'].min()
#     r = (grouped.dt.date.min(), grouped.dt.date.max())
#     time_range.append(r)


# # missing values before time aggregation TODO: do for heartrate only ig
# missing_before_agg = data.isna().sum().tolist()

# # missing values after time aggregation -> fill this in manually... 
# # automatic 14 for each except call & sms 
# # + 13 for activity 
# # + 1 for 'circumplex.arousal','circumplex.valence', 'mood' & -2 cuz there were like 2 days of measurements in the first 14 days? 
# # temp_data = fn.group_data(data, count=False)
# # missing_after_agg = temp_data.isna().sum().tolist()
# # missing_after_agg.insert(0, 0)


# missing_after_agg = [0 for i in range(len(data.columns))]
# for i, var in enumerate(data.columns):
#     if var == 'sms' or var == 'call' or var == 'id' or var == 'time': 
#         continue
#     if var == 'circumplex.arousal' or var == 'circumplex.valence' or var == 'mood':
#         missing_after_agg[i] += 1
#     missing_after_agg[i] += 14
#     if var == 'activity':
#         missing_after_agg[i] += 13



data_table = pd.DataFrame({'Variables': data.columns, 
                    # 'Data types': types, 
                    'number values': n_values, 
                    'number unique values': n_unique, 
                    # 'Mean (std)': mean_std,
                    # 'Value range': val_range,
                    # 'Date range': time_range, 
                    # 'Missing values (before time aggregation)': missing_before_agg,
                    # 'Missing values (after time aggregation)': missing_after_agg
                    })
data_table

,Variables,number values,number unique values
0,entry_num,18134,25
1,datetime,18134,18134
2,Time (s),18134,18134
3,X (m/s^2),18134,18131
4,Y (m/s^2),18134,18086
5,Z (m/s^2),18134,18111
6,Time (s).1,18134,18134
7,X (rad/s),18134,18133
8,Y (rad/s),18134,18134
9,Z (rad/s),18134,18132
